In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
#reading csvs from raw data sets
pop2020_df = pd.read_csv("../RawData/pop2020.csv")
pop2019_df=pd.read_csv("../RawData/pop2019.csv", encoding="ISO-8859-1")
inventory_df = pd.read_csv("../RawData/HomePriceFull.csv")
rental_df = pd.read_csv("../RawData/Rental_price.csv")

In [14]:
#cleaning popuation data sets (finding common cities from population data sets)
#selecting wanted data and renaming the columns from 2020 population data

pop2020_df = pop2020_df.iloc[:, 1:4]
pop2020_df=pop2020_df.rename(columns={"Name" : "City",
                                      "2020 Pop":"2020 Population"})
pop2020_df = pop2020_df.replace({"New York City": "New York"})

#selecting wanted data and renaming the columns from 2019 population data
columns = 'NAME', 'STNAME', 'POPESTIMATE2018', 'POPESTIMATE2019'
pop2019_df=pop2019_df.loc[:, columns]
pop2019_df=pop2019_df.rename(columns={"NAME" : "City",
                                      "STNAME" : "State",
                                      "POPESTIMATE2018": "2018 Population",
                                      "POPESTIMATE2019" : "2019 Population"})

#merging population data sets

pop2020_df['City'] = pop2020_df['City'] + ' city'
population_df = pd.merge(pop2019_df, pop2020_df, on=["City", "State"], how='inner')
duplicated = population_df.loc[population_df.duplicated(subset=["City", "State"]), "City"]
pop_df = population_df.drop(duplicated.index)
pop_df.reset_index(drop=True, inplace=True)
pop_df["City"] = pop_df["City"].str.replace("city", "")

#abbrev. state names
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

pop_df['State'] = pop_df['State'].map(us_state_abbrev).fillna(pop_df['State'])

#exporting data as csv file 
pop_df.to_csv("cleaned_population.csv")
pop_df

,City,State,2018 Population,2019 Population,2020 Population
0,Birmingham,AL,210487,209403,"208,940"
1,Huntsville,AL,198125,200574,"202,910"
2,Mobile,AL,189809,188720,"186,804"
3,Montgomery,AL,198899,198525,"197,282"
4,Chandler,AZ,257186,261165,"268,675"
...,...,...,...,...,...
177,Spokane,WA,219332,222081,"223,266"
178,Tacoma,WA,216111,217827,"222,603"
179,Vancouver,WA,182887,184463,"186,516"
180,Madison,WI,257854,259680,"264,030"


In [15]:
#renaming column name and selecting wanted data from inventory.csv
inventory_df = inventory_df.rename(columns={"RegionName":"City"})
inventory_df.drop(inventory_df.iloc[:, 5:124], inplace = True, axis = 1) 
inventory_df.drop(inventory_df.iloc[:, 0:2], inplace = True, axis = 1) 
del inventory_df["RegionType"]

#renaming columns and data
inventory_df['City'] = inventory_df['City'].str.split(',').str[0]
inventory_df = inventory_df.rename(columns={'StateName':'State'})
# inventory_df = inventory_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
#                                 "Dallas-Fort Worth":"Dallas",
#                                  "Miami-Fort Lauderdale":"Miami",
#                                  "Minneapolis-St Paul" : "Minneapolis",
#                                  "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
inventory_df = pd.merge(pop_df["City"], inventory_df, on="City", how="outer")
new_inventory_df=inventory_df.dropna(how="any")
new_inventory_df.reset_index(drop=True, inplace=True)
new_inventory_df.to_csv("cleaned_inventory.csv")
new_inventory_df

,City,State,9/30/05,10/31/05,11/30/05,12/31/05,1/31/06,2/28/06,3/31/06,4/30/06,...,12/31/19,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20
0,New York,NY,451754.0,458812.0,465566.0,472703.0,478395.0,483818.0,485937.0,487964.0,...,624329.0,624680.0,624947.0,624563.0,625025.0,625187.0,626609.0,627671.0,629936.0,632271.0
1,Los Angeles,CA,579066.0,589938.0,598654.0,605882.0,612398.0,617543.0,623699.0,627492.0,...,714109.0,718163.0,724618.0,731034.0,736655.0,738996.0,740780.0,747509.0,757047.0,768046.0
2,Houston,TX,128224.0,128018.0,128142.0,128090.0,128315.0,128326.0,128569.0,128707.0,...,188921.0,189055.0,189529.0,190455.0,191618.0,192630.0,193385.0,194246.0,195245.0,196902.0
3,Houston,MN,163351.0,166110.0,168560.0,169530.0,170155.0,170685.0,171387.0,172268.0,...,233741.0,233866.0,234005.0,234231.0,234077.0,234060.0,234177.0,234259.0,234393.0,234270.0
4,Houston,AR,56781.0,56983.0,57245.0,57583.0,58037.0,58458.0,58766.0,58927.0,...,75640.0,75708.0,76173.0,76081.0,76287.0,76330.0,77099.0,77906.0,79149.0,79858.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19021,Chidester,AR,30152.0,30087.0,29705.0,29340.0,28542.0,27663.0,27111.0,26938.0,...,31749.0,31605.0,31872.0,32221.0,32667.0,33247.0,33728.0,34385.0,34915.0,35162.0
19022,Burgin,KY,58660.0,57698.0,55892.0,54795.0,53873.0,53226.0,52367.0,51723.0,...,68591.0,68848.0,69266.0,69549.0,69784.0,69758.0,69741.0,69636.0,69767.0,70099.0
19023,Athalia,OH,50690.0,50948.0,51174.0,51393.0,51411.0,51629.0,51795.0,51962.0,...,79682.0,79588.0,79144.0,79266.0,79525.0,80134.0,80127.0,79905.0,80079.0,80083.0
19024,Cedar Valley,OK,200819.0,200861.0,201711.0,202213.0,202555.0,202500.0,202347.0,202705.0,...,260852.0,262321.0,262939.0,263146.0,263341.0,263217.0,263471.0,263767.0,264184.0,264500.0


In [16]:
#renaming column name and selecting wanted data from rental.csv
rental_df = rental_df.rename(columns={"RegionName":"City"})
rental_df.drop(rental_df.iloc[:, 2:50], inplace = True, axis = 1) 
del rental_df ["RegionID"]

#spliting state name form city and adding in a new column
rental_df["State"] = rental_df["City"].str.split(',').str[1]
rental_df['City'] = rental_df['City'].str.split(',').str[0]

#rearrnaging the order of the colums 
columns = ['City', 'State','2018-01','2018-02', '2018-03', '2018-04', '2018-05', '2018-06','2018-07', 
           '2018-08','2018-09', '2018-10', '2018-11', '2018-12','2019-01', '2019-02', '2019-03', 
           '2019-04', '2019-05', '2019-06','2019-07', '2019-08', '2019-09', '2019-10', '2019-11', 
           '2019-12','2020-01', '2020-02', '2020-03', '2020-04','2020-05', '2020-06','2020-07', 
           '2020-08', '2020-09']

rental_df=rental_df.loc[:, columns]

#renaming columns and data
rental_df = rental_df.replace({"Los Angeles-Long Beach-Anaheim": "Los Angeles",
                                "Dallas-Fort Worth":"Dallas",
                                 "Miami-Fort Lauderdale":"Miami",
                                 "Minneapolis-St Paul" : "Minneapolis",
                                 "Louisville-Jefferson County":"Louisville"})

#mering with common city list 
rental_df = pd.merge(pop_df["City"], rental_df, on="City", how="outer")
new_rental_df=rental_df.dropna(how="any")
new_rental_df.reset_index(drop=True, inplace=True)
new_rental_df.to_csv("cleaned_rental.csv")
new_rental_df

,City,State,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,...,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09
0,New York,NY,2598.0,2601.0,2605.0,2608.0,2612.0,2617.0,2621.0,2626.0,...,2698.0,2694.0,2686.0,2678.0,2671.0,2662.0,2653.0,2644.0,2635.0,2625.0
1,Los Angeles,CA,2380.0,2388.0,2397.0,2405.0,2414.0,2423.0,2432.0,2440.0,...,2542.0,2544.0,2545.0,2545.0,2546.0,2546.0,2546.0,2546.0,2545.0,2545.0
2,Chicago,IL,1705.0,1708.0,1710.0,1713.0,1716.0,1719.0,1722.0,1725.0,...,1778.0,1778.0,1777.0,1776.0,1776.0,1775.0,1774.0,1772.0,1771.0,1770.0
3,Dallas,TX,1470.0,1474.0,1478.0,1481.0,1485.0,1489.0,1493.0,1498.0,...,1560.0,1562.0,1563.0,1564.0,1566.0,1567.0,1568.0,1570.0,1571.0,1572.0
4,Philadelphia,PA,1505.0,1509.0,1512.0,1516.0,1521.0,1525.0,1530.0,1534.0,...,1598.0,1600.0,1602.0,1604.0,1606.0,1608.0,1610.0,1612.0,1614.0,1615.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Port St. Lucie,FL,1486.0,1495.0,1503.0,1512.0,1520.0,1529.0,1538.0,1545.0,...,1640.0,1648.0,1656.0,1664.0,1671.0,1679.0,1687.0,1695.0,1702.0,1710.0
101,Fort Collins,CO,1541.0,1545.0,1550.0,1555.0,1559.0,1564.0,1569.0,1574.0,...,1636.0,1638.0,1641.0,1644.0,1646.0,1649.0,1651.0,1653.0,1656.0,1658.0
102,Boulder,CO,1884.0,1890.0,1895.0,1901.0,1906.0,1911.0,1916.0,1921.0,...,1991.0,1992.0,1993.0,1995.0,1996.0,1997.0,1998.0,1998.0,1999.0,2000.0
103,Greeley,CO,1568.0,1574.0,1580.0,1586.0,1592.0,1598.0,1604.0,1611.0,...,1680.0,1682.0,1685.0,1688.0,1690.0,1693.0,1695.0,1698.0,1700.0,1703.0
